# Identify Rare Decay Phenomenon: τ → μμμ
This notebook tries to identify a supposed rare decay phenomenon of a subatomic particle: τ → μμμ. This decay has not or cannot be found in nature because it violates the law of conservation of energy. Nevertheless, in theory, it can exist. Therefore, the objective of this notebook is to maximize a weighted area under the receiver operating characterisitc curve (AUC or AUROC). 

AUROC has the true-positive rate (correctly classified as hit/decay) as some function of the false-positive rate (incorrectly classified as hit/decay): true-positive-rate = f(false-positive-rate). The best value is 1.0, and the worst value is 0.0. AUROC is used when one needs to be very sure that if a model says 'hit' that the instance is actually a hit.

In addition to maximizing the weighted AUROC, there are two modeling conditions: passing an agreement test and passing a correlation test. Because this phenomenon has not or cannot be found in nature, a Monte Carlo simulation (based on theory) generated data for instances of the rare decay phenomenon. These two tests are meant to ensure that a machine learning algorithm is not using the imperfections in the Monte Carlo simulation in its classification decisions.

This notebook is for a Kaggle competition, Flavours of Physics: Finding τ → μμμ (Kernels Only), which can be found here: https://www.kaggle.com/c/flavours-of-physics-kernels-only. Other functions that are drawn on the competition administrators' listed resources should have links attached.

In [1]:
import pandas
import numpy 


# Kaggle csv imports
# train = pandas.read_csv('../input/training.csv', index_col='id') # data on which to train models
# test = pandas.read_csv('../input/test.csv', index_col='id') # data on which final predictions will be made
# check_correlation = pandas.read_csv('../input/check_correlation.csv', index_col='id') # data on which to do correlation test
# check_agreement = pandas.read_csv('../input/check_agreement.csv', index_col='id') # data on which to do the agreement test

# My machine csv imports
train = pandas.read_csv('training.csv', index_col='id') # data on which to train models
test = pandas.read_csv('test.csv', index_col='id') # data on which final predictions will be made
check_correlation = pandas.read_csv('check_correlation.csv', index_col='id') # data on which to do correlation test
check_agreement = pandas.read_csv('check_agreement.csv', index_col='id') # data on which to do the agreement test

In [2]:
# Source: https://github.com/yandexdataschool/flavours-of-physics-start/blob/master/evaluation.py
from sklearn.metrics import roc_curve, auc


def __rolling_window(data, window_size):
    """
    Rolling window: take window with definite size through the array
    :param data: array-like
    :param window_size: size
    :return: the sequence of windows
    Example: data = array(1, 2, 3, 4, 5, 6), window_size = 4
        Then this function return array(array(1, 2, 3, 4), array(2, 3, 4, 5), array(3, 4, 5, 6))
    """
    shape = data.shape[:-1] + (data.shape[-1] - window_size + 1, window_size)
    strides = data.strides + (data.strides[-1],)
    return numpy.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)


def __cvm(subindices, total_events):
    """
    Compute Cramer-von Mises metric.
    Compared two distributions, where first is subset of second one.
    Assuming that second is ordered by ascending
    :param subindices: indices of events which will be associated with the first distribution
    :param total_events: count of events in the second distribution
    :return: cvm metric
    """
    target_distribution = numpy.arange(1, total_events + 1, dtype='float') / total_events
    subarray_distribution = numpy.cumsum(numpy.bincount(subindices, minlength=total_events), dtype='float')
    subarray_distribution /= 1.0 * subarray_distribution[-1]
    return numpy.mean((target_distribution - subarray_distribution) ** 2)


def compute_cvm(predictions, masses, n_neighbours=200, step=50):
    """
    Computing Cramer-von Mises (cvm) metric on background events: take average of cvms calculated for each mass bin.
    In each mass bin global prediction's cdf is compared to prediction's cdf in mass bin.
    :param predictions: array-like, predictions
    :param masses: array-like, in case of Kaggle tau23mu this is reconstructed mass
    :param n_neighbours: count of neighbours for event to define mass bin
    :param step: step through sorted mass-array to define next center of bin
    :return: average cvm value
    """
    predictions = numpy.array(predictions)
    masses = numpy.array(masses)
    assert len(predictions) == len(masses)

    # First, reorder by masses
    predictions = predictions[numpy.argsort(masses)]

    # Second, replace probabilities with order of probability among other events
    predictions = numpy.argsort(numpy.argsort(predictions, kind='mergesort'), kind='mergesort')

    # Now, each window forms a group, and we can compute contribution of each group to CvM
    cvms = []
    for window in __rolling_window(predictions, window_size=n_neighbours)[::step]:
        cvms.append(__cvm(subindices=window, total_events=len(predictions)))
    return numpy.mean(cvms)


def __roc_curve_splitted(data_zero, data_one, sample_weights_zero, sample_weights_one):
    """
    Compute roc curve
    :param data_zero: 0-labeled data
    :param data_one:  1-labeled data
    :param sample_weights_zero: weights for 0-labeled data
    :param sample_weights_one:  weights for 1-labeled data
    :return: roc curve
    """
    labels = [0] * len(data_zero) + [1] * len(data_one)
    weights = numpy.concatenate([sample_weights_zero, sample_weights_one])
    data_all = numpy.concatenate([data_zero, data_one])
    fpr, tpr, _ = roc_curve(labels, data_all, sample_weight=weights)
    return fpr, tpr


def compute_ks(data_prediction, mc_prediction, weights_data, weights_mc):
    """
    Compute Kolmogorov-Smirnov (ks) distance between real data predictions cdf and Monte Carlo one.
    :param data_prediction: array-like, real data predictions
    :param mc_prediction: array-like, Monte Carlo data predictions
    :param weights_data: array-like, real data weights
    :param weights_mc: array-like, Monte Carlo weights
    :return: ks value
    """
    assert len(data_prediction) == len(weights_data), 'Data length and weight one must be the same'
    assert len(mc_prediction) == len(weights_mc), 'Data length and weight one must be the same'

    data_prediction, mc_prediction = numpy.array(data_prediction), numpy.array(mc_prediction)
    weights_data, weights_mc = numpy.array(weights_data), numpy.array(weights_mc)

    assert numpy.all(data_prediction >= 0.) and numpy.all(data_prediction <= 1.), 'Data predictions are out of range [0, 1]'
    assert numpy.all(mc_prediction >= 0.) and numpy.all(mc_prediction <= 1.), 'MC predictions are out of range [0, 1]'

    weights_data /= numpy.sum(weights_data)
    weights_mc /= numpy.sum(weights_mc)

    fpr, tpr = __roc_curve_splitted(data_prediction, mc_prediction, weights_data, weights_mc)

    Dnm = numpy.max(numpy.abs(fpr - tpr))
    return Dnm


def roc_auc_truncated(labels, predictions, tpr_thresholds=(0.2, 0.4, 0.6, 0.8),
                      roc_weights=(4, 3, 2, 1, 0)):
    """
    Compute weighted area under ROC curve.
    :param labels: array-like, true labels
    :param predictions: array-like, predictions
    :param tpr_thresholds: array-like, true positive rate thresholds delimiting the ROC segments
    :param roc_weights: array-like, weights for true positive rate segments
    :return: weighted AUC
    """
    assert numpy.all(predictions >= 0.) and numpy.all(predictions <= 1.), 'Data predictions are out of range [0, 1]'
    assert len(tpr_thresholds) + 1 == len(roc_weights), 'Incompatible lengths of thresholds and weights'
    fpr, tpr, _ = roc_curve(labels, predictions)
    area = 0.
    tpr_thresholds = [0.] + list(tpr_thresholds) + [1.]
    for index in range(1, len(tpr_thresholds)):
        tpr_cut = numpy.minimum(tpr, tpr_thresholds[index])
        tpr_previous = numpy.minimum(tpr, tpr_thresholds[index - 1])
        area += roc_weights[index - 1] * (auc(fpr, tpr_cut, reorder=True) - auc(fpr, tpr_previous, reorder=True))
    tpr_thresholds = numpy.array(tpr_thresholds)
    # roc auc normalization to be 1 for an ideal classifier
    area /= numpy.sum((tpr_thresholds[1:] - tpr_thresholds[:-1]) * numpy.array(roc_weights))
    return area

In [3]:
# Based on the evaluation functions provided by the administrators of the 
# competition (found here: 
# https://github.com/yandexdataschool/flavours-of-physics-start/blob/master/baseline.ipynb)
def evaluate_model(model, variables, validation, check_agreement, check_correlation):
    """
    This function combines the evaluation of the agreement test (compute_ks()), the correlation
    test (compute_cvm()), and the area under the curve score (roc_auc_truncated()) for
    a given model.
    
    Assumptions: 
    i. all of the functions utilized herein have already been created
    ii. model has a method called predict_proba() (i.e., model is a classifier)
    iii. if the model requires any kind of scaling or has any other preprocessing,
    'model' should be a pipeline object that includes that preprocessing
    
    :param model: the already-fitted classification model or pipeline with the model
    :param variables: list of variables that the model uses in the same order 
        that the model expects them -- SHOULD NOT include target variable
    :param train: pandas dataframe of train.csv
    :param check_agreement: pandas dataframe of check_agreement.csv
    :param check_correlation: pandas dataframe of check_correlation.csv
    
    :returns: AUC if ALL tests passed; -1.0 * AUC otherwise
    """
    print('TEST RESULTS')
    
    # Agreement test
    agreement_probs = model.predict_proba(check_agreement[variables])[:, 1]

    ks = compute_ks(agreement_probs[check_agreement['signal'].values == 0],
                    agreement_probs[check_agreement['signal'].values == 1],
                    check_agreement[check_agreement['signal'] == 0]['weight'].values,
                    check_agreement[check_agreement['signal'] == 1]['weight'].values)

    ks_test_passed = ks < 0.09
    print ('KS metric: \t {0:.4f} < {1}? \t\t{2}.'.format(ks, 0.09, ks_test_passed)) # 0.09 specified by admin
    
    # Correlation test
    correlation_probs = model.predict_proba(check_correlation[variables])[:, 1]
    cvm = compute_cvm(correlation_probs, check_correlation['mass'])
    
    cv_test_passed = cvm < 0.002
    print('CvM metric: \t {0:.5f} < {1}? \t\t{2}.'.format(cvm, 0.002, cv_test_passed)) # 0.002 specified by admin
    
    # Weighted AUC on validation data
    validation_probs = model.predict_proba(validation[variables])[:, 1]
    auc = roc_auc_truncated(validation['signal'], validation_probs)
    
    auc_test_passed = auc > 0.834346386627 # 0.83-etc. comes from admin's baseline.ipynb 
    print('AUC: \t\t {0:.7f} > {1:.7f}? \t{2}.'.format(auc, 0.834346386627, auc_test_passed))
    # 0.834... comes from baseline.ipynb on admin github page
    
    # This is a bit like returning 2 things: + ==> all tests passed while
    # - ==> not all tests passed AND numpy.abs(AUC) is the model's AUC score
    # on validation data
    if auc_test_passed and cv_test_passed and ks_test_passed:
        return auc
    else: 
        return -1.0 * auc



Often times for classification problems, a good place to start is logistic regression. Therefore, the first model will just be a logistic regression using all of the variables.

In [4]:
# Columns that check_agreement.csv, check_correlation.csv, train.csv, and test.csv all have,
# minus the target variable: signal
unusable_columns = ['production', 'mass', 'min_ANNmuon', 'signal']
ubiquitous_columns = [col for col in train.columns.tolist() if col not in unusable_columns]

In [5]:
from sklearn.linear_model import LogisticRegressionCV

# A train/test split, sort of
validation = train.sample(frac=0.75, random_state=3) # train=25%; validation=75% -- reversed for training speed
train = train.drop(validation.index)

train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)

lrcv = LogisticRegressionCV(random_state=3)
lrcv.fit(train[ubiquitous_columns], train['signal'])

evaluate_model(lrcv, ubiquitous_columns, validation, check_agreement, check_correlation)

TEST RESULTS
KS metric: 	 0.1601 < 0.09? 		False.
CvM metric: 	 0.00081 < 0.002? 		True.
AUC: 		 0.9320031 > 0.8343464? 	True.


-0.9320031311142262

Well that didn't work. Let's just try each a logistic regression with each variable on its own. For each variable that passes all 3 tests, add it to a list so that the next logistic regressor can be trained on all of those columns whose individual logistic regressions passed all 3 tests.

In [6]:
# Make a list of columns that, when used in a logistic regression on
# their own, result in all tests being passed
lrcv_passed_columns = []
for col in ubiquitous_columns:
    lrcv = LogisticRegressionCV(random_state=3).fit(train[col].values.reshape(-1,1),
                                                    train['signal'])
    
    print(col)
    all_test_passed = evaluate_model(lrcv, [col],
                                     validation,
                                     check_agreement,
                                     check_correlation)
    print()
    
    # agreement and correlation tests passed AND AUC > baseline (the 0.83-etc. value)
    if all_test_passed > 0.0:
        lrcv_passed_columns.append(col)
   

LifeTime
TEST RESULTS
KS metric: 	 0.0823 < 0.09? 		True.
CvM metric: 	 0.00100 < 0.002? 		True.
AUC: 		 0.7680600 > 0.8343464? 	False.

dira
TEST RESULTS
KS metric: 	 0.0344 < 0.09? 		True.
CvM metric: 	 0.00128 < 0.002? 		True.
AUC: 		 0.9339494 > 0.8343464? 	True.

FlightDistance
TEST RESULTS
KS metric: 	 0.1074 < 0.09? 		False.
CvM metric: 	 0.00098 < 0.002? 		True.
AUC: 		 0.7498143 > 0.8343464? 	False.

FlightDistanceError
TEST RESULTS
KS metric: 	 0.0533 < 0.09? 		True.
CvM metric: 	 0.00091 < 0.002? 		True.
AUC: 		 0.7416115 > 0.8343464? 	False.

IP
TEST RESULTS
KS metric: 	 0.0735 < 0.09? 		True.
CvM metric: 	 0.00108 < 0.002? 		True.
AUC: 		 0.9505927 > 0.8343464? 	True.

IPSig
TEST RESULTS
KS metric: 	 0.0695 < 0.09? 		True.
CvM metric: 	 0.00096 < 0.002? 		True.
AUC: 		 0.9530628 > 0.8343464? 	True.

VertexChi2
TEST RESULTS
KS metric: 	 0.0231 < 0.09? 		True.
CvM metric: 	 0.00087 < 0.002? 		True.
AUC: 		 0.8429456 > 0.8343464? 	True.

pt
TEST RESULTS
KS metric: 	 0.0583 < 

In [7]:
lrcv = LogisticRegressionCV(random_state=3)
lrcv.fit(train[lrcv_passed_columns], train['signal'])

evaluate_model(lrcv, lrcv_passed_columns, validation, check_agreement, check_correlation)

TEST RESULTS
KS metric: 	 0.0576 < 0.09? 		True.
CvM metric: 	 0.00103 < 0.002? 		True.
AUC: 		 0.9709629 > 0.8343464? 	True.


0.9709628756266802

That's actually surprisingly good for logistic regression. I wonder if, in addition to these variables that passed the three tests in their own logistic regressions, there are other variables that could be added to improve weighted AUC without causing test failure. Let's see if there are any such variables.

In [8]:
def iteratively_add_cols(model, base_columns, ubiquitous_columns,
                         train, validation, check_agreement, check_correlation):
    """
    Identify candidate columns (those that improve weighted AUC and still pass all tests),
    and iteratively add candidate columns until no longer improve AUC or cause test failure
    
    :param model: initialized model for which the optimal list of columns will be compliled
    :param train: X and y together, training data
    :param validation: X and y together, validation data
    :param check_agreement: ...
    :param check_correlation: ...
    :param base_columns: list of columns that will definitely be used for the final model
    :param ubiquitous_columns: all possible columns that a model could use
    
    :returns: list of columns from ubquitious columns that includes base_columsn
    as well as others that maximize weighted AUC while still passing all tests
    """

    model.fit(train[base_columns], train['signal'])

    print('Model Baseline')
    # Baseline logistic regression cv AUC by which models with additional columns will be judged
    current_auc = evaluate_model(model, base_columns, validation,
                                 check_agreement, check_correlation)

    candidate_columns = [] 
    unused_columns = [col for col in ubiquitous_columns if col not in base_columns]
    print('Finding candidates to add')
    for col in unused_columns:

        model.fit(train[base_columns+[col]], train['signal'])

        print()
        print(col)
        extra_column_auc = evaluate_model(model, base_columns+[col], validation,
                                          check_agreement, check_correlation)
        auc_improvement = extra_column_auc - current_auc

        # would only be positive if extra_column_auc is positive AND if adding variable did 
        # improve regression
        if auc_improvement > 0.0:
            candidate_columns.append( (col, auc_improvement) )

    print(candidate_columns)
    # Just to be safe, iteratively add all candidate columns to the model
    # but stop when test failure

    # list of candidate columns ordered from greatest to least AUC improvement
    # https://stackoverflow.com/questions/8459231/sort-tuples-based-on-second-parameter
    print('Evaluating candidates')
    if candidate_columns is not None:
        candidate_columns.sort(key=lambda tupe: tupe[1], reverse=True)
        auc_maximizing_columns = [col for col in base_columns]
        best_auc_so_far = current_auc # baseline AUC

        for col_tupe in candidate_columns:
            col = col_tupe[0]

            print()
            print(col)
            model.fit(train[auc_maximizing_columns+[col]], train['signal'])
            extra_column_auc = evaluate_model(model, auc_maximizing_columns+[col], validation,
                                              check_agreement, check_correlation)
            auc_improvement = extra_column_auc - current_auc

            # if adding the ith varible doesn't cause tests failure
            # AND if it still improves AUC
            if auc_improvement > 0.0 and extra_column_auc > best_auc_so_far:
                auc_maximizing_columns.append(col)
                best_auc_so_far = extra_column_auc

    # else there are no candidate columns to evaluate
    else:
        auc_maximizing_columns = [col for col in base_columns]
    print('Done')
    
    return auc_maximizing_columns


In [9]:
# lrcv_auc_maximizing_columns = iteratively_add_cols(lrcv, lrcv_passed_columns, ubiquitous_columns, 
#                                                    train, validation, check_agreement, check_correlation)

# from a previous iteration of the above
lrcv_auc_maximizing_columns = ['dira',
                                 'IP',
                                 'IPSig',
                                 'VertexChi2',
                                 'ISO_SumBDT',
                                 'p0_IsoBDT',
                                 'p1_IsoBDT',
                                 'p2_IsoBDT',
                                 'p0_track_Chi2Dof',
                                 'IP_p0p2',
                                 'iso',
                                 'p1_track_Chi2Dof',
                                 'isolationd',
                                 'isolationf',
                                 'p2_track_Chi2Dof',
                                 'FlightDistanceError',
                                 'isolationb',
                                 'DOCAone']

In [10]:
# Let's see what the improvements were
lrcv = LogisticRegressionCV(random_state=3)
lrcv.fit(train[lrcv_auc_maximizing_columns], train['signal'])

evaluate_model(lrcv, lrcv_auc_maximizing_columns, validation, check_agreement, check_correlation)


TEST RESULTS
KS metric: 	 0.0461 < 0.09? 		True.
CvM metric: 	 0.00107 < 0.002? 		True.
AUC: 		 0.9788274 > 0.8343464? 	True.


0.9788273977619022

That's an improvement over the logistic regression before those columns were added. Previously the weighted AUROC was 0.97096-ish. This seemingly indicates that the random forest will perform better on the lrcv auc maximizing columns (relative to its performance on lrcv passed columns) as well.

In [11]:
from sklearn.ensemble import RandomForestClassifier

# RFC with fewer estimators so that iteratively_add_cols() goes faster
rfc = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=3)
# check if the random forest can use more columns that the logistic regression
# could not use
# rfc_auc_maximizing_columns = iteratively_add_cols(rfc, lrcv_auc_maximizing_columns, ubiquitous_columns, 
#                                                   train, validation, check_agreement, check_correlation)

# from a prior run of the above
rfc_auc_maximizing_columns = ['dira',
                                 'IP',
                                 'IPSig',
                                 'VertexChi2',
                                 'ISO_SumBDT',
                                 'p0_IsoBDT',
                                 'p1_IsoBDT',
                                 'p2_IsoBDT',
                                 'p0_track_Chi2Dof',
                                 'IP_p0p2',
                                 'iso',
                                 'p1_track_Chi2Dof',
                                 'isolationd',
                                 'isolationf',
                                 'p2_track_Chi2Dof',
                                 'FlightDistanceError',
                                 'isolationb',
                                 'DOCAone',
                                 'p0_IP',
                                 'p2_pt',
                                 'p1_IP',
                                 'p2_eta',
                                 'LifeTime',
                                 'pt',
                                 'p2_IPSig']

rfc = RandomForestClassifier(n_estimators=2000, max_depth=None, random_state=3)
rfc.fit(train[lrcv_auc_maximizing_columns], train['signal'])
evaluate_model(rfc, lrcv_auc_maximizing_columns, validation, check_agreement, check_correlation)
print()


TEST RESULTS
KS metric: 	 0.0760 < 0.09? 		True.
CvM metric: 	 0.00102 < 0.002? 		True.
AUC: 		 0.9843816 > 0.8343464? 	True.



In [12]:
# Make the final (probability) predictions
# result = pandas.DataFrame({'id': test.index})
# result['prediction'] = rfc.predict_proba(test[lrcv_auc_maximizing_columns])[:, 1]

# result.to_csv('rfc_with_logistic_selection_pt986.csv', index=False, sep=',')

The random forest did well, though not as well as it was expected to do: its weighted AUC was 0.978899. This put it at 6th place out of 15 teams, which is not that bad. At the very least, we have a personal baseline score to beat.

Let's keep going and see what a neural network can do.

Let's see what a neural network can do.

In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

mlpc = MLPClassifier(hidden_layer_sizes=[50],
                     activation='tanh',
                     solver='adam',
                     learning_rate_init=1e-4, # improved AUC by 0.002-ish
                     random_state=3)

# mlpc_auc_maximizing_columns = iteratively_add_cols(mlpc, lrcv_auc_maximizing_columns, ubiquitous_columns, 
#                                                    train, validation, check_agreement, check_correlation)

# from a previous run of the above
mlpc_auc_maximizing_columns = ['dira',
                                 'IP',
                                 'IPSig',
                                 'VertexChi2',
                                 'ISO_SumBDT',
                                 'p0_IsoBDT',
                                 'p1_IsoBDT',
                                 'p2_IsoBDT',
                                 'p0_track_Chi2Dof',
                                 'IP_p0p2',
                                 'iso',
                                 'p1_track_Chi2Dof',
                                 'isolationd',
                                 'isolationf',
                                 'p2_track_Chi2Dof',
                                 'FlightDistanceError',
                                 'isolationb',
                                 'DOCAone',
                                 'FlightDistance',
                                 'p0_IPSig',
                                 'p2_IPSig',
                                 'p1_eta',
                                 'p0_eta']

mlpc = MLPClassifier(hidden_layer_sizes=[200],
                     activation='tanh',
                     solver='adam',
                     learning_rate_init=1e-4, # improved AUC by 0.002-ish
                     random_state=3)
mlpc.fit(train[mlpc_auc_maximizing_columns], train['signal'])

print('...on training data')
evaluate_model(mlpc, mlpc_auc_maximizing_columns, train, check_agreement, check_correlation)

print('...on validation data')
evaluate_model(mlpc, mlpc_auc_maximizing_columns, validation, check_agreement, check_correlation)
print()

...on training data
TEST RESULTS
KS metric: 	 0.0445 < 0.09? 		True.
CvM metric: 	 0.00086 < 0.002? 		True.
AUC: 		 0.9865038 > 0.8343464? 	True.
...on validation data
TEST RESULTS
KS metric: 	 0.0445 < 0.09? 		True.
CvM metric: 	 0.00086 < 0.002? 		True.
AUC: 		 0.9844195 > 0.8343464? 	True.



The logistic regression cv, random forest classifier, and multilayer perceptron each individually seem to perform pretty well. I wonder what happens when their probabilities are averaged. Will that help or hurt the weighted AUROC? Will it cause test failure?

In [14]:
from sklearn.ensemble import VotingClassifier

vc = VotingClassifier(estimators=[('rfc', RandomForestClassifier(n_estimators=50, random_state=3)),
                                  ('lrcv', LogisticRegressionCV(random_state=3)),
                                  ('mlpc', MLPClassifier(hidden_layer_sizes=[50],
                                                         activation='tanh',
                                                         solver='adam',
                                                         learning_rate_init=1e-4,
                                                         random_state=3))],
                                  voting='soft')

# vc_auc_maximizing_columns = iteratively_add_cols(vc, lrcv_auc_maximizing_columns, ubiquitous_columns, 
#                                                  train.sample(frac=0.02), validation, check_agreement, check_correlation)

# from a previous run of the above
vc_auc_maximizing_columns = ['dira',
                             'IP',
                             'IPSig',
                             'VertexChi2',
                             'ISO_SumBDT',
                             'p0_IsoBDT',
                             'p1_IsoBDT',
                             'p2_IsoBDT',
                             'p0_track_Chi2Dof',
                             'IP_p0p2',
                             'iso',
                             'p1_track_Chi2Dof',
                             'isolationd',
                             'isolationf',
                             'p2_track_Chi2Dof',
                             'FlightDistanceError',
                             'isolationb',
                             'DOCAone',
                             'p0_IP']

vc = VotingClassifier(estimators=[('rfc', RandomForestClassifier(n_estimators=200, random_state=3)),
                                  ('lrcv', LogisticRegressionCV(random_state=3)),
                                  ('mlpc', MLPClassifier(hidden_layer_sizes=[200],
                                                         activation='tanh',
                                                         solver='adam',
                                                         learning_rate_init=1e-4,
                                                         random_state=3))],
                                  voting='soft')

vc.fit(train[vc_auc_maximizing_columns], train['signal'])

evaluate_model(vc, vc_auc_maximizing_columns, validation, check_agreement, check_correlation)


TEST RESULTS
KS metric: 	 0.0540 < 0.09? 		True.
CvM metric: 	 0.00100 < 0.002? 		True.
AUC: 		 0.9846739 > 0.8343464? 	True.


0.98467386646591

It seems that averaging the predicted probabilities helped slightly (with an emphasis on "slightly"). Overall, though, I'm not sure if averaging their predicted probabilities will reduce bias or introduce bias. 

Let's try the support vector classifier (SVC).

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# These values were narrowed down by previous grid searches
svc_param_dict = {'C': numpy.arange(0.7, 1.2, 0.1),
                  'kernel': ['rbf'],
                  'tol': [1e-4]}

svc_gridsearchcv = GridSearchCV(estimator=SVC(), 
                                param_grid=svc_param_dict,
                                verbose=10)

svc_ss = StandardScaler().fit(pandas.concat([train[lrcv_auc_maximizing_columns],
                                             validation[lrcv_auc_maximizing_columns],]))

num_samples = 10000
sample_indicies = train.sample(n=num_samples).index

# easier 
svc_gridsearchcv.fit(svc_ss.transform(train[lrcv_auc_maximizing_columns].iloc[sample_indicies].values),
                     train['signal'].iloc[sample_indicies])

print('best estimator', svc_gridsearchcv.best_estimator_)
print('best score', svc_gridsearchcv.best_score_)

svc_pipeline = Pipeline( [('ss', StandardScaler()),
                          ('svc', SVC(C=1.0,
                                      kernel='rbf',
                                      tol=1e-4,
                                      probability=True))] ) # prob=True slows training down considerably
# according to scikit learn page and just running this once

svc_pipeline.fit(train[lrcv_auc_maximizing_columns], train['signal'])
evaluate_model(svc_pipeline, lrcv_auc_maximizing_columns, validation, check_agreement, check_correlation)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] C=0.7, kernel=rbf, tol=0.0001 ...................................
[CV]  C=0.7, kernel=rbf, tol=0.0001, score=0.856628674265147, total=   1.8s
[CV] C=0.7, kernel=rbf, tol=0.0001 ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s


[CV]  C=0.7, kernel=rbf, tol=0.0001, score=0.8485302939412117, total=   1.7s
[CV] C=0.7, kernel=rbf, tol=0.0001 ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.0s remaining:    0.0s


[CV]  C=0.7, kernel=rbf, tol=0.0001, score=0.8427370948379351, total=   1.8s
[CV] C=0.7999999999999999, kernel=rbf, tol=0.0001 ....................


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   22.0s remaining:    0.0s


[CV]  C=0.9999999999999999, kernel=rbf, tol=0.0001, score=0.855128974205159, total=   1.7s
[CV] C=0.9999999999999999, kernel=rbf, tol=0.0001 ....................
[CV]  C=0.9999999999999999, kernel=rbf, tol=0.0001, score=0.8494301139772046, total=   1.7s
[CV] C=0.9999999999999999, kernel=rbf, tol=0.0001 ....................
[CV]  C=0.9999999999999999, kernel=rbf, tol=0.0001, score=0.842436974789916, total=   1.7s
[CV] C=1.0999999999999999, kernel=rbf, tol=0.0001 ....................
[CV]  C=1.0999999999999999, kernel=rbf, tol=0.0001, score=0.855128974205159, total=   1.7s
[CV] C=1.0999999999999999, kernel=rbf, tol=0.0001 ....................
[CV]  C=1.0999999999999999, kernel=rbf, tol=0.0001, score=0.8497300539892022, total=   1.7s
[CV] C=1.0999999999999999, kernel=rbf, tol=0.0001 ....................
[CV]  C=1.0999999999999999, kernel=rbf, tol=0.0001, score=0.8418367346938775, total=   1.6s


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   36.4s finished


best estimator SVC(C=0.7, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.0001, verbose=False)
best score 0.8493
TEST RESULTS
KS metric: 	 0.0801 < 0.09? 		True.
CvM metric: 	 0.00098 < 0.002? 		True.
AUC: 		 0.9813031 > 0.8343464? 	True.


0.9813031429748271

Given that the support vector classifier fails the agreement test and given that its AUC is not much better than the random forest, there is no advantage in keeping it alone. Perphaps if it is part of a voting classifier it would not affect the voting classifier such that the voting classifier would fail tests. In that case, perhaps the SVC would provide some marginal benefit.

In [16]:
from sklearn.ensemble import VotingClassifier

vc = VotingClassifier(estimators=[('rfc', RandomForestClassifier(n_estimators=50, random_state=3)),
                                  ('lrcv', LogisticRegressionCV(random_state=3)),
                                  ('mlpc', MLPClassifier(hidden_layer_sizes=[50],
                                                         activation='tanh',
                                                         solver='adam',
                                                         learning_rate_init=1e-4,
                                                         random_state=3)),
                                 ('svc_pipeline',Pipeline( [('ss', StandardScaler()),
                                                             ('svc', SVC(C=1.0,
                                                                          kernel='rbf',
                                                                          tol=1e-4,
                                                                          probability=True))] ))],
                                  voting='soft')
# vc_auc_maximizing_columns = iteratively_add_cols(vc, lrcv_auc_maximizing_columns, ubiquitous_columns, 
#                                                  train.sample(n=2000), validation, check_agreement, check_correlation)

# from a previous run of the above
vc_auc_maximizing_columns = ['dira',
                             'IP',
                             'IPSig',
                             'VertexChi2',
                             'ISO_SumBDT',
                             'p0_IsoBDT',
                             'p1_IsoBDT',
                             'p2_IsoBDT',
                             'p0_track_Chi2Dof',
                             'IP_p0p2',
                             'iso',
                             'p1_track_Chi2Dof',
                             'isolationd',
                             'isolationf',
                             'p2_track_Chi2Dof',
                             'FlightDistanceError',
                             'isolationb',
                             'DOCAone',
                             'pt',
                             'IP_p1p2',
                             'p1_eta',
                             'p2_IPSig']

vc = VotingClassifier(estimators=[('rfc', RandomForestClassifier(n_estimators=200, random_state=3)),
                                  ('lrcv', LogisticRegressionCV(random_state=3)),
                                  ('mlpc', MLPClassifier(hidden_layer_sizes=[200],
                                                         activation='tanh',
                                                         solver='adam',
                                                         learning_rate_init=1e-4,
                                                         random_state=3)),
                                 ('svc_pipeline',Pipeline( [('ss', StandardScaler()),
                                                            ('svc', SVC(C=1.0,
                                                                        kernel='rbf',
                                                                        tol=1e-4,
                                                                        probability=True))] ))],
                      voting='soft')

vc.fit(train[vc_auc_maximizing_columns], train['signal'])

evaluate_model(vc, vc_auc_maximizing_columns, validation, check_agreement, check_correlation)

TEST RESULTS
KS metric: 	 0.0746 < 0.09? 		True.
CvM metric: 	 0.00096 < 0.002? 		True.
AUC: 		 0.9852558 > 0.8343464? 	True.


0.9852557964676435

A bit surprisingly, the SVC that failed the agreement test on its, when included in the voting classifier, improved AUC! It improved AUC slightly over the 3-model voting classifier (by about 0.0009), and the voting classifier still passed all of the tests, though the agreement test (KS metric) is a bit closer to the max than I would like. We'll go with this model as the next model to submit to Kaggle.

In [17]:
# Make the final (probability) predictions
# result = pandas.DataFrame({'id': test.index})

# print('making probability predictions')
# result['prediction'] = vc.predict_proba(test[lrcv_auc_maximizing_columns])[:, 1]

# print('writing to csv')
# result.to_csv('vc_with_logistic_selection_pt9876.csv', index=False, sep=',')
# print('done!')

Unfortunately, that voting classifier with the support vector classifier performed worse than expected. It scored a weighted AUC score of 0.980854 on Kaggle's public leaderboard. While this is an improvement over the random forest by itself, it is not comparable to the validation weighted AUC score: 0.9853.

Given all of the discrepancies between the expected scores and the actual, public leaderboard scores, what is needed most is cross validation. 

In [20]:
from sklearn.model_selection import ShuffleSplit
import scipy.stats as stats
import math

def auc_confidence_interval_calc(model, df_with_x_and_y, input_cols, target_col, alpha=0.99, validation_size=0.997):
    """
    Print alpha*100% confidence interval for a model's weighted AUC.
    
    :param model: initialized but not fitted classification model
    :param alpha: alpha for the confidence interval (default is 0.99, so gives 99% CI)
    :param df_with_x_and_y: pandas dataframe with inputs and target cols together
    :param input_cols: list of input columns
    :param target_cols: string that is the column name of the label column
    :param validation_size: size of validation split; proportion of df_with_x_and_y that
    will be used to obtain AUC score  (0.997 gets 99% CI's containing the 2 Kaggle public
    leaderboard scores from the rfc and vc models: 0.978899 and 0.980854 respectively. 
    Moreover, the avereage is roughly the same as the public leaderboard score.)
    
    :returns: list of each validation AUC score for each iteration
    """
    
    # 0.3%=train/99.7%=test
    # can you calculate that test_size percentage?
    # plus, confidence intervals?
    # plus, less is more#
    # formula for test_size?
    ss = ShuffleSplit(n_splits=20, test_size=validation_size) # based on 1st rfc public score

    val_auc_scores = []
    iteration = 1
    for train_indicies, val_indicies in ss.split(df_with_x_and_y[input_cols], df_with_x_and_y[target_col]):
        model.fit(df_with_x_and_y[input_cols].iloc[train_indicies],
                  df_with_x_and_y[target_col].iloc[train_indicies])

            # Weighted AUC on validation data
        validation_probs = model.predict_proba(df_with_x_and_y[input_cols].iloc[val_indicies])[:, 1]
        val_auc = roc_auc_truncated(df_with_x_and_y[target_col].iloc[val_indicies], validation_probs)

        val_auc_scores.append(val_auc)
        print('AUC on iteration {0}: {1:.7f}'.format(iteration, val_auc))
        iteration += 1

    print()
    avg_auc = numpy.mean(val_auc_scores)
    print('Average AUC score: {0:.7f}'.format(avg_auc))

    sdev_auc = numpy.std(val_auc_scores)
    print('Standard deviation of AUC scores: {0:.7f}'.format(sdev_auc))

    # Assume t-distribution for distribution 
    # https://stackoverflow.com/questions/28242593/correct-way-to-obtain-confidence-interval-with-scipy
    ci = stats.t.interval(df=len(val_auc_scores)-1,
                              alpha=alpha,
                              loc=avg_auc,
                              scale=(sdev_auc / math.sqrt(len(val_auc_scores))), )
    print('{}% Confidence interval for average AUC score:   {}'.format(alpha*100.0, ci))
    
    return val_auc_scores

In [21]:
all_train = pandas.read_csv('training.csv', index_col='id') # data on which to train models

rfc = RandomForestClassifier(n_estimators=200)
auc_confidence_interval_calc(rfc, all_train, lrcv_auc_maximizing_columns, 'signal')
print()

AUC on iteration 1: 0.9790934
AUC on iteration 2: 0.9794377
AUC on iteration 3: 0.9787227
AUC on iteration 4: 0.9782271
AUC on iteration 5: 0.9771244
AUC on iteration 6: 0.9774839
AUC on iteration 7: 0.9777520
AUC on iteration 8: 0.9785983
AUC on iteration 9: 0.9754350
AUC on iteration 10: 0.9775475
AUC on iteration 11: 0.9781546
AUC on iteration 12: 0.9792799
AUC on iteration 13: 0.9790953
AUC on iteration 14: 0.9769444
AUC on iteration 15: 0.9794860
AUC on iteration 16: 0.9787367
AUC on iteration 17: 0.9774344
AUC on iteration 18: 0.9782153
AUC on iteration 19: 0.9788952
AUC on iteration 20: 0.9765295

Average AUC score: 0.9781097
Standard deviation of AUC scores: 0.0010462
99.0% Confidence interval for average AUC score:   (0.9774404059016386, 0.9787789212211475)



In [22]:
vc = VotingClassifier(estimators=[('rfc', RandomForestClassifier(n_estimators=200, random_state=3)),
                                  ('lrcv', LogisticRegressionCV(random_state=3)),
                                  ('mlpc', MLPClassifier(hidden_layer_sizes=[200],
                                                         activation='tanh',
                                                         solver='adam',
                                                         learning_rate_init=1e-4,
                                                         random_state=3)),
                                 ('svc_pipeline',Pipeline( [('ss', StandardScaler()),
                                                             ('svc', SVC(C=1.0,
                                                                          kernel='rbf',
                                                                          tol=1e-4,
                                                                          probability=True))] ))],
                                  voting='soft')

auc_confidence_interval_calc(vc, all_train, vc_auc_maximizing_columns, 'signal')
print()

AUC on iteration 1: 0.9811112
AUC on iteration 2: 0.9814783
AUC on iteration 3: 0.9794880
AUC on iteration 4: 0.9776859
AUC on iteration 5: 0.9810987
AUC on iteration 6: 0.9826247
AUC on iteration 7: 0.9825912
AUC on iteration 8: 0.9818502
AUC on iteration 9: 0.9746417
AUC on iteration 10: 0.9820130
AUC on iteration 11: 0.9812954
AUC on iteration 12: 0.9798816
AUC on iteration 13: 0.9805154
AUC on iteration 14: 0.9821510
AUC on iteration 15: 0.9806148
AUC on iteration 16: 0.9798040
AUC on iteration 17: 0.9799202
AUC on iteration 18: 0.9828073
AUC on iteration 19: 0.9796643
AUC on iteration 20: 0.9770145

Average AUC score: 0.9804126
Standard deviation of AUC scores: 0.0020068
99.0% Confidence interval for average AUC score:   (0.979128787118357, 0.9816963676078468)



After tuning the function auc_confidence_interval_calc(), it now gives a 99% confidence interval for the average AUC score. That average AUC score, at least for the random forest classifier (rfc) and the 4-model voting classifier (vc), is roughly the same as the Kaggle public leaderboard score. This will be a very useful tool to have when evaluating future models. I wonder how the other, 3-model voting classifier measures up to the 4-model voting classifier based on this function?

In [23]:
vc = VotingClassifier(estimators=[('rfc', RandomForestClassifier(n_estimators=200, random_state=3)),
                                  ('lrcv', LogisticRegressionCV(random_state=3)),
                                  ('mlpc', MLPClassifier(hidden_layer_sizes=[200],
                                                         activation='tanh',
                                                         solver='adam',
                                                         learning_rate_init=1e-4,
                                                         random_state=3))],
                                  voting='soft')
auc_confidence_interval_calc(vc, all_train, lrcv_auc_maximizing_columns, 'signal')
print()

AUC on iteration 1: 0.9790614


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


AUC on iteration 2: 0.9801280


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


AUC on iteration 3: 0.9753712


...

AUC on iteration 14: 0.9825593
AUC on iteration 15: 0.9790026
AUC on iteration 16: 0.9798670


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


AUC on iteration 17: 0.9780733
AUC on iteration 18: 0.9768961


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


AUC on iteration 19: 0.9804116
AUC on iteration 20: 0.9779095

Average AUC score: 0.9789758
Standard deviation of AUC scores: 0.0014411
99.0% Confidence interval for average AUC score:   (0.9780538474227752, 0.9798976980629083)



Not only does the 3-model one seem like it would be worse than the 4-model one, it would also not be much better than the random forest classifier on its own. 

In [24]:
# ...

Although these tiles were cut, I did try principal componenet and linear discriminant analysis with a neural network, random forest, and SVC. I also tried AdaBoost, GradientBoost, and DecisionTree classifiers by themselves and within the voting classifier. For the voting classifier, I also tried to mess with the model weights, but unfortunately, still, nothing really improved the weighted AUROC. 

Because of this and in the interest of moving on to another competition, this is where I will leave off on this physics competition. The best model, the voting classifier, scored 0.980854
for its weighted AUC, thereby placing it at, currently, 5th out of 20 teams (top 25%).